# 【練習問題】自動車の走行距離予測

In [284]:
# モジュールの読み込み
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error as MSE

# Googleマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [285]:
# データの読み込み
data_train = pd.read_csv("/content/SIGNATE/train.tsv", sep="\t")
X_test = pd.read_csv("/content/SIGNATE/test.tsv", sep="\t")

In [286]:
data_train.head()

,id,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,0,29.0,4,135.0,84.00,2525.0,16.0,82,1,dodge aries se
1,3,31.9,4,89.0,71.00,1925.0,14.0,79,2,vw rabbit custom
2,9,19.0,6,156.0,108.0,2930.0,15.5,76,3,toyota mark ii
3,11,28.0,4,90.0,75.00,2125.0,14.5,74,1,dodge colt
4,13,37.7,4,89.0,62.00,2050.0,17.3,81,3,toyota tercel


In [287]:
data_train.describe()

,id,mpg,cylinders,displacement,weight,acceleration,model year,origin
count,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000
mean,200.170854,24.307035,5.296482,183.311558,2883.839196,15.647739,76.165829,1.582915
std,113.432759,7.797665,1.644562,98.400457,819.766870,2.701885,3.802928,0.798932
min,0.000000,9.000000,3.000000,71.000000,1613.000000,8.500000,70.000000,1.000000
25%,98.500000,18.000000,4.000000,98.000000,2217.500000,14.000000,73.000000,1.000000
50%,202.000000,24.000000,4.000000,140.000000,2702.000000,15.500000,76.000000,1.000000
75%,294.500000,30.500000,6.000000,250.000000,3426.500000,17.150000,80.000000,2.000000
max,396.000000,44.600000,8.000000,454.000000,5140.000000,23.700000,82.000000,3.000000


In [288]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            199 non-null    int64  
 1   mpg           199 non-null    float64
 2   cylinders     199 non-null    int64  
 3   displacement  199 non-null    float64
 4   horsepower    199 non-null    object 
 5   weight        199 non-null    float64
 6   acceleration  199 non-null    float64
 7   model year    199 non-null    int64  
 8   origin        199 non-null    int64  
 9   car name      199 non-null    object 
dtypes: float64(4), int64(4), object(2)
memory usage: 15.7+ KB


In [289]:
print(data_train['horsepower'].unique())
print(X_test['horsepower'].unique())

['84.00' '71.00' '108.0' '75.00' '62.00' '90.00' '105.0' '180.0' '113.0'
 '135.0' '120.0' '88.00' '165.0' '100.0' '69.00' '70.00' '60.00' '80.00'
 '132.0' '85.00' '130.0' '93.00' '?' '175.0' '68.00' '58.00' '110.0'
 '65.00' '67.00' '78.00' '48.00' '112.0' '86.00' '83.00' '81.00' '82.00'
 '92.00' '97.00' '76.00' '193.0' '148.0' '72.00' '170.0' '140.0' '220.0'
 '79.00' '198.0' '150.0' '64.00' '96.00' '138.0' '52.00' '145.0' '95.00'
 '160.0' '74.00' '54.00' '149.0' '139.0' '129.0' '63.00' '122.0' '200.0'
 '46.00' '153.0' '89.00' '155.0' '61.00' '215.0' '190.0' '115.0']
['76.00' '98.00' '92.00' '110.0' '88.00' '90.00' '150.0' '105.0' '190.0'
 '75.00' '175.0' '65.00' '87.00' '77.00' '140.0' '67.00' '48.00' '170.0'
 '155.0' '84.00' '70.00' '60.00' '95.00' '215.0' '52.00' '180.0' '71.00'
 '49.00' '68.00' '63.00' '103.0' '85.00' '80.00' '152.0' '58.00' '79.00'
 '86.00' '78.00' '?' '100.0' '69.00' '145.0' '230.0' '125.0' '115.0'
 '130.0' '198.0' '94.00' '153.0' '83.00' '208.0' '46.00' '137.0' '

In [290]:
#条件にマッチしたIndexを取得
drop_index = data_train.index[data_train['horsepower'] == '?']
#条件にマッチしたIndexを削除
data_train = data_train.drop(drop_index)

data_train['horsepower'] = data_train['horsepower'].astype(float)

mean = data_train['horsepower'].mean()
#条件にマッチしたIndexを置換
X_test['horsepower'] = X_test['horsepower'].replace('?', mean)

In [291]:
X_test['horsepower'] = X_test['horsepower'].astype(float)

In [292]:
print(data_train.info())
print(X_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195 entries, 0 to 198
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            195 non-null    int64  
 1   mpg           195 non-null    float64
 2   cylinders     195 non-null    int64  
 3   displacement  195 non-null    float64
 4   horsepower    195 non-null    float64
 5   weight        195 non-null    float64
 6   acceleration  195 non-null    float64
 7   model year    195 non-null    int64  
 8   origin        195 non-null    int64  
 9   car name      195 non-null    object 
dtypes: float64(5), int64(4), object(1)
memory usage: 16.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            199 non-null    int64  
 1   cylinders     199 non-null    int64  
 2   displacement  199 non-null    

In [293]:
data_train = data_train.drop(columns=['id', 'model year', 'origin', 'car name'])
X_test = X_test.drop(columns=['id', 'model year', 'origin', 'car name'])

In [294]:
X_train = data_train[['cylinders',	'displacement',	'horsepower',	'weight',	'acceleration']]
y_train = data_train['mpg']

In [295]:
lr = LR()

In [296]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [297]:
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

In [298]:
mse_train = MSE(y_train, y_pred_train)

In [299]:
rmse_train = np.sqrt(mse_train)
print(rmse_train)

4.192614685478882


In [302]:
sub = pd.read_csv('/content/SIGNATE/sample_submit.csv', encoding = 'UTF-8', names=['id', 'ans'])
sub['ans'] = list(map(int, y_pred_test))
sub.to_csv("/content/SIGNATE/Car_Mileage_Prediction_submit01.csv", header=False, index=False)